In [13]:
import matplotlib.pyplot as plt
import pandas as pd


In [3]:
df = pd.read_csv('reddit_randomness_12k.csv')
df.head()

,subreddit,response_code
0,changemyview,302
1,Terraform,302
2,lostpause,302
3,USPS,302
4,MaliciousCompliance,302


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12077 entries, 0 to 12076
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subreddit      12077 non-null  object
 1   response_code  12077 non-null  object
dtypes: object(2)
memory usage: 188.8+ KB


In [6]:
for col in df.columns:
  print(col, 'has', len(df[col].unique()), 'unique values')


subreddit has 3679 unique values
response_code has 2 unique values


In [7]:
df['response_code'].unique()

array(['302', 'response_code'], dtype=object)

In [8]:
'Error' in df['subreddit']

False

In [20]:
from collections import Counter

c = Counter(df['subreddit'])
for sub in df['subreddit'][:10]:
  print(c[sub], sub)

8 changemyview
5 Terraform
5 lostpause
4 USPS
4 MaliciousCompliance
6 BATProject
2 obs
7 apple
8 IndoorGarden
3 Dimension20


In [21]:
c.most_common(30)

[('subreddit', 22),
 ('elderscrollsonline', 10),
 ('bridezillas', 10),
 ('AMDHelp', 10),
 ('Polska', 10),
 ('RepLadies', 10),
 ('viralphotoleaks', 10),
 ('Noom', 10),
 ('ROBLOXExploiting', 10),
 ('inkarnate', 9),
 ('ContraPoints', 9),
 ('linuxmint', 9),
 ('DHgate', 9),
 ('CampingGear', 9),
 ('Adguard', 9),
 ('AskMen', 9),
 ('nonononoyes', 9),
 ('madnesscombat', 9),
 ('czech', 9),
 ('LeagueOfMemes', 9),
 ('DayzXbox', 9),
 ('Bedbugs', 9),
 ('Optifine', 9),
 ('changemyview', 8),
 ('IndoorGarden', 8),
 ('Discord_Bots', 8),
 ('FedEx', 8),
 ('halifax', 8),
 ('NCAAFBseries', 8),
 ('rpcs3', 8)]

In [22]:
# Try using /about and /about/traffic to get more info on the subreddits
# about/traffic claims to need a mod OAuth

import json
import requests

In [29]:
headers = {
    'User-Agent': 'Mozilla/5.0'
}

r = requests.get('https://old.reddit.com/r/subreddit/', headers=headers)
r.status_code

200

In [30]:
r.text

'<!doctype html><html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en"><head><title>/r/subreddit: Share a subreddit!</title><meta name="keywords" content=" reddit, reddit.com, vote, comment, submit " /><meta name="description" content="/r/subreddit --- Come here to share an unrecognized subreddit or to find a subreddit you\'ve been looking for! ___." /><meta name="referrer" content="always"><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><link type="application/opensearchdescription+xml" rel="search" href="/static/opensearch.xml"/><link rel="canonical" href="https://www.reddit.com/r/subreddit/" /><meta name="viewport" content="width=1024"><link rel="dns-prefetch" href="//out.reddit.com"><link rel="preconnect" href="//out.reddit.com"><meta property="og:image" content="https://www.redditstatic.com/new-icon.png"><meta property="og:site_name" content="reddit"><meta property="og:description" content="/r/subreddit --- Come here to share an unrecognized subred

In [31]:
len(r.text)

230261

In [39]:
idx = r.text.find('subscribers')
idx

21064

In [44]:
r.text[idx-50:idx+300]

'ion remove" href="#">leave</a></span><span class="subscribers"><span class="number">16,600</span>&#32;<span class="word">readers</span></span><p class="users-online" title="users viewing this subreddit in the past 15 minutes"><span class="number">27</span>&#32;<span class="word">users here now</span></p><form action="#" class="usertext warn-on-unlo'

In [41]:
!pip install beautifulsoup4

In [42]:
from bs4 import BeautifulSoup

In [43]:
soup = BeautifulSoup(r.text, 'html.parser')

In [54]:
sub_attrs = {'class': 'subscribers'}
total_subs = soup.find_all(attrs=sub_attrs)
print(type(total_subs))
total_subs

<class 'bs4.element.ResultSet'>


[<span class="subscribers"><span class="number">16,600</span> <span class="word">readers</span></span>]

In [53]:
user_attrs = {'title': 'users viewing this subreddit in the past 15 minutes'}
soup.find_all(attrs=user_attrs)

[<p class="users-online" title="users viewing this subreddit in the past 15 minutes"><span class="number">27</span> <span class="word">users here now</span></p>]

In [59]:
type(total_subs[0])

bs4.element.Tag

In [67]:
num_attr = {'class':'number'}
exact_subs = total_subs[0].find(attrs=num_attr)
exact_subs

<span class="number">16,600</span>

In [71]:
int(exact_subs.getText().replace(',',''))

16600

In [129]:
def get_subs_and_users(subreddit_name):
  """
  GETs the front page of the subredit

  returns the current number of subscribers
  and the number of users who have visited in the past 15 minutes
  """
  sub_url = 'https://old.reddit.com/r/' + subreddit_name
  headers = {'User-Agent': 'Mozilla/5.0'}
  r = requests.get(sub_url, headers=headers)
  if r.status_code != 200:
    raise RuntimeError('Received status code ' + r.status_code + ' tyring to GET ' + subreddit_name )
  
  try:
    soup = BeautifulSoup(r.text, 'html.parser')

    subs = soup.find(attrs={'class': 'subscribers'})
    subs = subs.find(attrs={'class': 'number'})
    total_subs = int(subs.getText().replace(',',''))

    users = soup.find(attrs={'class': 'users-online'})
    users = users.find(attrs={'class': 'number'})
    cur_users = int(users.getText().replace(',',''))

    return total_subs, cur_users

  except AttributeError:
    return np.nan, np.nan

In [130]:
get_subs_and_users('pornhwa')

(nan, nan)

In [97]:
import numpy as np

from time import sleep, time
from tqdm import tqdm

In [131]:
#start_idxs = [0, 1000, 2000, 3000]
# There was a nsfw sub that crashed the scraper
# so restart where it left off
start_idxs = [2905]

for start_idx in start_idxs:
  
  iterable = tqdm(df['subreddit'].unique()[start_idx:])

  extra_data = {}
  for sub in iterable:
    try:
      subs, users = get_subs_and_users(sub)
    except RuntimeError:
      subs, users = np.nan, np.nan
    extra_data[sub] = {'subscribers': subs, 'current_users': users, 'accessed': time()}
    #sleep(2)

  # This makes the subreddit column the index
  # reset the index and rename the resulting column
  extra_df = pd.DataFrame.from_dict(extra_data, orient='index')
  extra_df = extra_df.reset_index()
  extra_df = extra_df.rename(columns={'index':'subreddit'})

  extra_df.to_csv('extra_data_' + str(start_idx), index=False)

100%|██████████| 774/774 [11:35<00:00,  1.11it/s]


In [122]:
len(extra_data.keys())

905

In [123]:
extra_df = pd.DataFrame.from_dict(extra_data, orient='index')
extra_df = extra_df.reset_index()
extra_df = extra_df.rename(columns={'index':'subreddit'})
extra_df.head()

,subreddit,subscribers,current_users,accessed
0,thinkorswim,31521,126,1.632086e+09
1,wine_gaming,33729,57,1.632086e+09
2,OculusQuest,272367,1889,1.632086e+09
3,kodi,108816,141,1.632086e+09
4,TurkeyJerky,169205,276,1.632086e+09


In [124]:
  extra_df.to_csv('extra_data_2000', index=False)

In [127]:
'pornhwa' in c.keys()

True

In [128]:
c['pornhwa']

3

In [148]:
extra = pd.read_csv('extra_data.csv')
extra.head()

,subreddit,subscribers,current_users,accessed
0,changemyview,1349762,1272,1632083284.7689114
1,Terraform,19623,38,1632083285.6234503
2,lostpause,78056,255,1632083286.6756592
3,USPS,69497,568,1632083287.6244717
4,MaliciousCompliance,1664837,5484,1632083288.4133828


In [149]:
extra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3682 entries, 0 to 3681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subreddit      3682 non-null   object
 1   subscribers    3682 non-null   object
 2   current_users  3682 non-null   object
 3   accessed       3682 non-null   object
dtypes: object(4)
memory usage: 115.2+ KB


In [150]:
extra.loc[extra['subscribers'] == 'subscribers']

,subreddit,subscribers,current_users,accessed
1000,subreddit,subscribers,current_users,accessed
2001,subreddit,subscribers,current_users,accessed
2907,subreddit,subscribers,current_users,accessed


In [151]:
extra_clean = extra.drop(index=[1000,2001,2907])

In [152]:
extra_clean.iloc[998:1002]

,subreddit,subscribers,current_users,accessed
998,fatestaynight,164265,944,1632084225.2922947
999,Frenemies2,8712,798,1632084226.3839796
1001,sony,39371,312,1632084833.0385523
1002,psg,25915,1447,1632084833.9963012


In [153]:
extra_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3679 entries, 0 to 3681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subreddit      3679 non-null   object
 1   subscribers    3679 non-null   object
 2   current_users  3679 non-null   object
 3   accessed       3679 non-null   object
dtypes: object(4)
memory usage: 143.7+ KB


In [158]:
extra_clean = extra_clean.astype({'subscribers':float, 'current_users':float})
extra_clean = extra_clean.astype({'subscribers':int, 'current_users':int})
extra_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3679 entries, 0 to 3681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subreddit      3679 non-null   object
 1   subscribers    3679 non-null   int64 
 2   current_users  3679 non-null   int64 
 3   accessed       3679 non-null   object
dtypes: int64(2), object(2)
memory usage: 143.7+ KB


In [159]:
extra_clean.to_csv('extra_clean.csv', index=False)

In [160]:
randomness = pd.read_csv('reddit_randomness_12k.csv')
randomness.head()

,subreddit,response_code
0,changemyview,302
1,Terraform,302
2,lostpause,302
3,USPS,302
4,MaliciousCompliance,302


In [161]:
randomness.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12077 entries, 0 to 12076
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subreddit      12077 non-null  object
 1   response_code  12077 non-null  object
dtypes: object(2)
memory usage: 188.8+ KB


In [162]:
randomness.loc[randomness['response_code']=='response_code']

,subreddit,response_code
1000,subreddit,response_code
2001,subreddit,response_code
2007,subreddit,response_code
2013,subreddit,response_code
2019,subreddit,response_code
2025,subreddit,response_code
2031,subreddit,response_code
2037,subreddit,response_code
2043,subreddit,response_code
2049,subreddit,response_code


In [163]:
idx_to_drop = randomness.loc[randomness['response_code']=='response_code'].index
randomness_clean = randomness.drop(index=idx_to_drop)
randomness_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12055 entries, 0 to 12076
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subreddit      12055 non-null  object
 1   response_code  12055 non-null  object
dtypes: object(2)
memory usage: 282.5+ KB


In [164]:
randomness_clean.iloc[998:1002]

,subreddit,response_code
998,Terraria,302
999,DragonMaid,302
1001,ireland,302
1002,videography,302


In [166]:
randomness_clean = randomness_clean.reset_index()
randomness_clean.iloc[998:1002]

,index,subreddit,response_code
998,998,Terraria,302
999,999,DragonMaid,302
1000,1001,ireland,302
1001,1002,videography,302


In [167]:
randomness_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12055 entries, 0 to 12054
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          12055 non-null  int64 
 1   subreddit      12055 non-null  object
 2   response_code  12055 non-null  object
dtypes: int64(1), object(2)
memory usage: 282.7+ KB


In [168]:
randomness_clean = randomness_clean.drop(columns='index')
randomness_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12055 entries, 0 to 12054
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subreddit      12055 non-null  object
 1   response_code  12055 non-null  object
dtypes: object(2)
memory usage: 188.5+ KB


In [169]:
randomness_clean = randomness_clean.astype({'response_code':int})
randomness_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12055 entries, 0 to 12054
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subreddit      12055 non-null  object
 1   response_code  12055 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 188.5+ KB


In [170]:
randomness_clean.to_csv('randomness_clean.csv')